# Database creation

In [2]:
import sqlite3
import pandas as pd
import os

In [ ]:
# Define the paths to form the QSL database
base_path = "./" 
folders = ['hosp', 'icu', 'note']
db_path = '../database/mimic.db'

# Connect to SQLite database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Iterate through each folder and process .csv.gz files
for folder in folders: 
    folder_path = os.path.join(base_path, folder)
    
    for file in os.listdir(folder_path):
        if file.endswith(".csv.gz"):
            file_path = os.path.join(folder_path, file)
            table_name = file.replace(".csv.gz", "")

            chunk_size = 1000000
            for chunk in pd.read_csv(file_path, compression='gzip', chunksize= chunk_size):
                chunk.to_sql(table_name, conn, if_exists='append', index=False)
            
            print(f'Finished processing {file} into table {table_name}')

# Close the connection
conn.close()

## Add indexes to the database

In [3]:

# Connect to the database
conn = sqlite3.connect('../database/mimic.db')
cursor = conn.cursor()

# Get all table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Iterate through each table
for table in tables:
    table_name = table[0]  # Extract the table name from the tuple
    cursor.execute(f"PRAGMA table_info({table_name});")
    columns = cursor.fetchall()
    
    # Get list of column names
    column_names = [column[1] for column in columns]
    
    # Check for subject_id and create index if it exists
    if "subject_id" in column_names:
        index_name = f"idx_{table_name}_subject_id"
        try:
            cursor.execute(f"CREATE INDEX {index_name} ON {table_name} (subject_id);")
            print(f"Created index {index_name} on {table_name}")
        except sqlite3.OperationalError as e:
            print(f"Skipped {index_name}: {e}")
    
    # Check for hadm_id and create index if it exists
    if "hadm_id" in column_names:
        index_name = f"idx_{table_name}_hadm_id"
        try:
            cursor.execute(f"CREATE INDEX {index_name} ON {table_name} (hadm_id);")
            print(f"Created index {index_name} on {table_name}")
        except sqlite3.OperationalError as e:
            print(f"Skipped {index_name}: {e}")
    
    # Check if neither subject_id nor hadm_id exists
    if "subject_id" not in column_names and "hadm_id" not in column_names:
        print(f"Table {table_name} has neither subject_id nor hadm_id")

# Commit changes and close the connection
conn.commit()
conn.close()

Created index idx_poe_subject_id on poe
Created index idx_poe_hadm_id on poe
Table d_hcpcs has neither subject_id nor hadm_id
Created index idx_poe_detail_subject_id on poe_detail
Created index idx_patients_subject_id on patients
Created index idx_diagnoses_icd_subject_id on diagnoses_icd
Created index idx_diagnoses_icd_hadm_id on diagnoses_icd
Created index idx_emar_detail_subject_id on emar_detail
Table provider has neither subject_id nor hadm_id
Skipped idx_prescriptions_subject_id: index idx_prescriptions_subject_id already exists
Created index idx_prescriptions_hadm_id on prescriptions
Created index idx_drgcodes_subject_id on drgcodes
Created index idx_drgcodes_hadm_id on drgcodes
Table d_icd_diagnoses has neither subject_id nor hadm_id
Table d_labitems has neither subject_id nor hadm_id
Created index idx_transfers_subject_id on transfers
Created index idx_transfers_hadm_id on transfers
Created index idx_admissions_subject_id on admissions
Created index idx_admissions_hadm_id on a